In [2]:
using Primes

In [3]:
#algoritmo extendido de Euclides
function XMDC(a,b)
    x0, x = 1, 0; y0, y = 0, 1
    
    # nós trabalhamos com números positivos, mas aceitamos argumentos negativos
    sign_a, sign_b = 1, 1
    if a < 0 
        a = -a
        sign_a = -1
    end
    
    if b < 0 
        b = -b
        sign_b = -1
    end
    
    
    #x0*a + y0*b = a
    #x*a + y*b = b
    
    while b>0 
        q, r = a ÷ b, a%b
        a, b = b, r
        
        #atualizar x, x0, y, y0
        x, x0 = x0 - q*x, x
        y, y0 = y0 - q*y, y
    end 
    
    return a, sign_a*x0, sign_b*y0
end

XMDC (generic function with 1 method)

In [4]:
function ExpModN( a, e, n )
    A, P::BigInt, E = a, 1, e
        
    while E != 0        
        D = E % 2 # the last binary digit of E         
        if D == 1
            P = (A*P) % n
            E = (E-1)÷2
        else
            E = E÷2
        end
        A = (A*A) % n
    end 
    return P
end

ExpModN (generic function with 1 method)

In [5]:
function IsPseudoPrimeInBase( n, b )
    return ExpModN( b, n-1, n ) == 1
end

IsPseudoPrimeInBase (generic function with 1 method)

In [6]:
IsPseudoPrimeInBase( 95, 2 )

false

In [7]:
for i in (2:10000)
    if !isprime( i ) & IsPseudoPrimeInBase( i, 5 ) 
        print( i, " " )
    end
end

4 124 217 561 781 1541 1729 1891 2821 4123 5461 5611 5662 5731 6601 7449 7813 8029 8911 9881 

In [8]:
function TesteFermat( n )
    
    d = 2*floor(log( n ))+1
    for b in (2:d)
        if !IsPseudoPrimeInBase( n, b )
            return "COMPOSTO"
        end
    end
    return "PROVAVELMENTE PRIMO"
end

TesteFermat (generic function with 1 method)

In [9]:
for i in (2:1_000_000)
    if !isprime( i ) & (TesteFermat( i ) == "PROVAVELMENTE PRIMO") 
        print( i, " " )
    end
end

252601 294409 399001 410041 488881 512461 

In [10]:
function TesteMiller( n, b )
    
    q = n-1
    k = 0
    while q % 2 == 0
        q = q//2
        k = k+1
    end
    
    r = ExpModN( b, q, n )
    if r == 1
        return "PROVAVELMENTE PRIMO"
    end
    
    for i in (0:k-1)
        if r == n-1
            return "PROVAVELMENTE PRIMO"
        end
        r = r*r % n
    end
    
    return "COMPOSTO"  
end

TesteMiller (generic function with 1 method)

In [11]:
for i in (2:1_000_000)
    if !isprime( i ) & (TesteMiller( i, 3 ) == "PROVAVELMENTE PRIMO") 
        print( i, " " )
    end
end

121 286 703 1891 3281 8401 8911 10585 12403 16531 18721 19345 23521 24046 31621 44287 47197 55969 63139 74593 79003 82513 87913 88573 97567 105163 111361 112141 148417 152551 182527 188191 211411 218791 221761 226801 228073 232726 282133 288163 293401 313447 320167 327781 328021 340033 359341 364231 365713 385003 385201 399001 432821 443713 453259 478297 497503 504913 512461 551881 563347 625873 638731 655051 658711 679057 709873 801139 823213 859951 867043 876961 894781 951481 973241 994507 

In [12]:
function TesteMillerBach( n )
        
    k = min(n-1,floor(2*log( n )^2))
    for b in (2:k)
        if TesteMiller( n, b ) == "COMPOSTO"
            return "COMPOSTO"
        end
    end
        
    return "PRIMO (HGR)" #Hipótese generalizada de Riemann
end

TesteMillerBach (generic function with 1 method)

In [13]:
for i in (2:100_000)
    if !isprime( i ) & (TesteMillerBach( i ) == "PRIMO (HGR)") 
        print( i, " " )
    end
end

In [15]:
function TesteMillerRabin( n )
        
    k = floor(log( n )) + 1
    for i in (1:k)
        b = rand( 2:n-2 )
        if TesteMiller( n, b ) == "COMPOSTO"
            return "COMPOSTO"
        end
    end
    
    return "PROVAVELMENTE PRIMO"
end

TesteMillerRabin (generic function with 1 method)

In [16]:
for i in (5:10_000_000)
    if !isprime( i ) & (TesteMillerRabin( i ) == "PROVAVELMENTE PRIMO") 
        print( i, " " )
    end
end

InterruptException: InterruptException:

In [17]:
p = BigInt(2)^2203-1 #primo de Mersenne
q = BigInt(2)^2281-1 #outro primo de Mersenne
n = p*q
@time TesteMillerBach( n )
@time isprime( n )
#@time factor( n )

  0.268906 seconds (712.63 k allocations: 34.075 MiB, 4.82% gc time)
  0.043811 seconds (1 allocation: 35.516 KiB)


false

In [19]:
function TesteLucasLehmer( p )
    m = BigInt( 2 )^p-1
    s = 4
    
    for i in (1:p-2) 
        s = (s*s-2) % m
    end
        
    if s == 0 
        return "PRIMO"
    else
        return "COMPOSTO"
    end
end

TesteLucasLehmer (generic function with 1 method)

In [20]:
TesteLucasLehmer( 13  )

"PRIMO"

In [43]:
F(k) = BigInt(2)^(2^k)+1

F (generic function with 1 method)

In [44]:
F(5)

4294967297

In [46]:
m = 1
F(5) % (1+10*64)

0

In [26]:
# Criptografia RSA
# Computação de Bob
println( TesteLucasLehmer( 521 )) 
println( TesteLucasLehmer( 607 )) 
p = BigInt(2)^607-1; q = BigInt(2)^521-1
n = p*q
phin = (p-1)*(q-1)
global d = 0
while true 
    global e = rand( 2:(n-2) )
    mdc, d, _ = XMDC( e, phin ); 
    if mdc == 1
        break     
    end
end

d = d % phin
if d < 0
    d = d+phin
end

PRIMO
PRIMO


In [27]:
# funções de codificação e descodificação 
C( b ) = ExpModN( b, e, n ) #b^e mod n
D( c ) = ExpModN( c, d, n ) #c^d mod n

D (generic function with 1 method)

In [28]:
# mensagem descodificada da Alice
b = rand( 2:n-1 )
b

3417549090337600387756113151291309206000926679470301767667827882932285757602021731471918572390672273280977506537321801016497456501876912862389522831408164277164410779601240628783561212234923744498394454726476648845336623695625281887860306651626299328017128342718098050183095279667389721526960932517622564849299793273875577943999856437676742

In [29]:
# mensagem criprografada da Alice
c = C( b )

2729865247358777562052585688184714760591306946316759504199982523403312501944794971106222004258911061033994254415270391896958772277393569623121502212498304531504224383722501122832114727200523203574340729666450973810644639710525780092205612850856138018340431442693221864426353963864865267668598047474802981510534856393065201692317151128793806

In [30]:
# Bob aplica a função D na mensagem recebida 
b1 = D( c )
# Bob obtem a mensagem original da Alice
b1 == b


true

In [31]:
n = 10403
e = 8743
p, q = factor( n )
phin = (p[1]-1)*(q[1]-1)
phin
_, d = XMDC( e, phin )
@assert d*e % phin == 1
C( b ) = ExpModN( b, e, n )
D( c ) = ExpModN( c, d, n )

D (generic function with 1 method)

In [32]:
msg = [ 4746, 8214, 9372, 9009, 4453, 8198 ]
[ D(x) for x in msg ]


6-element Array{BigInt,1}:
 1514
 2722
 1029
 9931
 1831
   14

In [33]:
# Criptografia de Rabin
p = BigInt(1000000000000000000000000000099)
q = BigInt(10000000000000000000000000000000000000139)
@assert p % 4 == 3 || q % 4 == 3 
n = p*q
_, yp, yq = XMDC( p, q )

yp = yp % n

yq = yq % n

yp*p+yq*q

1

In [34]:
function DecodeRabin( c )
    mp = ExpModN( c, (p+1)//4, p )
    mq = ExpModN( c, (q+1)//4, q )
    r1 = (yp*p*mq+yq*q*mp) % n
    r2 = (yp*p*mq-yq*q*mp) % n 
    return r1, r2, n-r1, n-r2 
end

DecodeRabin (generic function with 1 method)

In [35]:
b = rand( 1:(n-1) )
c = b^2 % n
r1, r2, r3, r4 = DecodeRabin( c )
@assert r1^2 % n == c && r2^2 % n == c && r3^2 % n == c && r4^2 % n == c 

In [37]:
# Troca de chaves de Diffie-Hellman
# computação comum
p = 38450217703680880490141016889392805287264761398712883307976597757938978857702527567193907088613428174327748769779175270772048436148553483274190345656562325735211741290766697605370338848672044225288837
g = rand( 2:(p-1) )

8203252554977833726365289042709512876622462056521923575291723112672132096050255389732507362955199304718403179555476310744203961211981777588908552564900852306436941827091301356988182460220791046642883

In [39]:
# Computação da Alice
a = rand( 2:(p-1) )
x = ExpModN( g, a, p )

11270892406909116453123306940282663248979916799618915758300148436983286344978867653576268250716010306623240344038963283362203320699963953316882655777481533442447175695398616201673004844609506408714449

In [40]:
# Computação do Bob
b = rand( 2:(p-1) )
y = ExpModN( g, b, p )

30064825016404229398693762926641210950612536627328901206685508085708634960858206245740409970114353674987128785789236663246404688503882613456400793424338940002112266653604324456630960915061238061883635

In [41]:
# Computação da Alice 
ExpModN( y, a, p )

28956597302366977293341463675669405833855148616441090355203848227845352592913535448185194346159941182384536678839808085908436855740062416072151199702171990208854956824492301555447262250355559582314754

In [42]:
# Computação do Bob 
ExpModN( x, b, p )

28956597302366977293341463675669405833855148616441090355203848227845352592913535448185194346159941182384536678839808085908436855740062416072151199702171990208854956824492301555447262250355559582314754